In [1]:
import pandas as pd
import numpy as np
name_bms1_csv = "Dataset Paper/BMS1.csv"
name_bms2_csv = "Dataset Paper/BMS2.csv"

In [22]:
# compute CSV file BMS1
file = open("Dataset Paper/BMS1_spmf.txt","r")
read = file.readlines()
lines = list()
for line in read:
    line = line.replace("-2","")
    line = line.replace(" ","")
    line = line[:-2]
    line = line.rstrip("-")
    line= line.replace("-1",",")    
    lines.append(line)
file.close()
file_write = open(name_bms1_csv,"w")
for line in lines:
    file_write.write(line+"\n")
file_write.close()

In [2]:
# BMS1
my_cols = [x for x in range(0,268)]
df = pd.read_csv(name_bms1_csv,names=my_cols,index_col=None ,engine='c',dtype='object')
df = df.where((pd.notnull(df)), None)
df_dummies = pd.get_dummies(df)
# sum item with same number after _
columns = list()
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1] and column.split('_')[1] not in columns ):
            columns.append(column.split('_')[1])

# remove duplicate
columns = list(set(columns))  
zero_data = np.zeros(shape=(len(df_dummies),len(columns)))
df_new = pd.DataFrame(zero_data, columns=columns,dtype='uint8')
# qualcosa non val alcuni sono valori maggiori di 1
# create a transactional data
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1]):
            if(column.split('_')[0]=='0'):
                colonna_new  = column.split('_')[1]
                if(len(df_new.index[df_new[colonna_new] > 0].tolist())==0):
                    df_new[colonna_new]=df_new[colonna_new] + df_dummies[column] + df_dummies[column_1]
                else:
                    df_new[colonna_new] = df_new[colonna_new] + df_dummies[column_1]
                #print(df_new[colonna_new])
                
# ora bisogn aaggiungere len(df) -len(columns) item finti con vlori tutti 0 
# per avere una matrice quadrata (vedi paper)
len(columns)
# write item in a file
with open("Dataset Paper/lista_items_BMS1.txt","w") as f:
    for x in list(df_new.columns):
        f.write(str(x)+"\n")
        
df_new.to_csv("Dataset Paper/dataBMS1_transiction.csv")


In [12]:
#READ FILE BMS2
file = open("Dataset Paper/BMS2.txt","r")
read = file.readlines()
lines = list()
for line in read:
    line = line.replace("-2","")
    line = line.replace(" ","")
    line = line[:-2]
    line = line.rstrip("-")
    line= line.replace("-1",",")
    lines.append(line)
file.close()
file_write = open(name_bms2_csv,"w")
for line in lines:
    file_write.write(line+"\n")
file_write.close()

In [33]:
# BMS2
my_cols = [x for x in range(0,161)]
df = pd.read_csv(name_bms2_csv,names=my_cols,index_col=None ,engine='c',dtype= 'object')
df = df.where((pd.notnull(df)), None)
df_dummies = pd.get_dummies(df)
# sum item with same number after _
columns = list()
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1] and column.split('_')[1] not in columns ):
            columns.append(column.split('_')[1])

# remove duplicate
columns = list(set(columns))  
zero_data = np.zeros(shape=(len(df_dummies),len(columns)))
df_new = pd.DataFrame(zero_data, columns=columns,dtype='uint8')
# qualcosa non val alcuni sono valori maggiori di 1
# create a transactional data
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1]):
            if(column.split('_')[0]=='0'):
                colonna_new  = column.split('_')[1]
                if(len(df_new.index[df_new[colonna_new] > 0].tolist())==0):
                    df_new[colonna_new]=df_new[colonna_new] + df_dummies[column] + df_dummies[column_1]
                else:
                    df_new[colonna_new] = df_new[colonna_new] + df_dummies[column_1]
                #print(df_new[colonna_new])
                
# ora bisogn aaggiungere len(df) -len(columns) item finti con vlori tutti 0 
# per avere una matrice quadrata (vedi paper)
len(columns)
# write item in a file
with open("Dataset Paper/lista_items_BMS2.txt","w") as f:
    for x in list(df_new.columns):
        f.write(str(x)+"\n")
        
df_new.to_csv("Dataset Paper/dataBMS2_transiction.csv")

KeyboardInterrupt: 

In [ ]:
#DATAFRAME COMPLET --> matrix square
zero_data_to_add = np.zeros(shape=(len(df_dummies),len(df_dummies)-len(columns)))
columns_to_add = [x for x in range(0,len(df_dummies)-len(columns))] 
df_to_add = pd.DataFrame(zero_data_to_add, columns=columns_to_add,dtype='uint8')

# creo il dataset completo aggiungendo tutti gli zeri che mancano
df_new_complete = pd.concat([df_new, df_to_add], axis=1)
# rename all column name
df_new_complete.columns = [i for i in range(df_new_complete.shape[1])]
# for write in a file uncommented this line
# save dataset complete in 1000x1000

# df_new_complete.to_csv("data_complete.csv",header = False, index=False)

# compute RCM for matrix complete
sparse_complete = csr_matrix(df_new_complete)
# RCM algorithm fro df_tuncate
order_complete = reverse_cuthill_mckee(sparse_complete)

# print(df_new_complete)
# print(len(df_new_complete),len(order_complete))

print("df_new_complete without RCM")
plt.spy(df_new_complete)
plt.show()
print("df_new_complete with RCM")
plt.spy(df_new_complete.iloc[order_complete][order_complete])
plt.show()

[i,j] = np.where(df_new_complete.iloc[order_complete][order_complete] == 1)
bw = max(i-j) +1
print("bw after RCM " + str(bw))
[i,j] = np.where(df_new_complete == 1)
bw = max(i-j) +1
print("bw first RCM " +str(bw))
df_completed_reordered = df_new_complete.iloc[order_complete][order_complete]